In [ ]:
from connector import spark_connector

In [ ]:
import pandas as pd
import numpy as np
from pyspark.sql import Window
import pyspark.sql.functions as F
import pyspark.sql.types as T

In [ ]:
from sklearn.neighbors import NearestNeighbors
import functools
import itertools

In [ ]:
spark_connector?

In [ ]:
con = spark_connector(exec_inst=100, exec_cores=5, memory_exec=25, enable_graphframes=True)

In [ ]:
con.spark

In [ ]:
from sklearn.datasets import make_blobs

In [ ]:
X, y, c = make_blobs(n_samples=150000, n_features=2, centers=75, cluster_std=0.75, center_box=(-120.0, 120.0), shuffle=True, random_state=0, return_centers=True)

In [ ]:
data = pd.DataFrame(X, columns=[f'x{i+1}' for i in range(X.shape[1])])
data['y'] = y
data = data.merge(pd.DataFrame(c, columns=[f'centroids_{i+1}' for i in range(X.shape[1])]).reset_index(), left_on='y', right_on='index', how='left').drop('index', axis=1)

In [ ]:
data

In [ ]:
import seaborn as sns

In [ ]:
sns.scatterplot(x=data.x1, y=data.x2, hue=data.y)

In [ ]:
data = data.rename({'y':'index_train'}, axis=1).drop(['centroids_1','centroids_2'], axis=1)

In [ ]:
train_df = con.spark.createDataFrame(data)

In [ ]:
from HierarchicalClustering import HiClust

In [ ]:
HiClust?

In [ ]:
clust = HiClust(con.spark, train_df=train_df)

In [ ]:
clust.fit(window_size=None, normalize='zscore')

In [ ]:
clust.predict(metric = 'euclidean',distance_threshold=0.5, weighted_first_iter=True)

In [ ]:
clust.hist

In [ ]:
clust.hist.select('cluster_id_4').distinct().count()

In [ ]:
con.stop_spark()